In [1]:
import pandas as pd
pd.set_option('display.max_columns', 999)
PATH = "../../DS-Unit-2-Applied-Modeling/data/dota/"

In [66]:
%%time
matches_small_chunk = pd.read_csv(PATH + 'matches_small.csv', chunksize=10000)
i = 0

for chunk in matches_small_chunk:
    if i == 0:
        df = chunk[chunk.human_players == 10]
        df.to_csv(path_or_buf=PATH + "full_players_games.csv", mode='w', index=False)
        i += df.shape[0]
        continue
    df = chunk[chunk.human_players == 10]
    df.to_csv(path_or_buf=PATH + "full_players_games.csv", mode='a', header=False, index=False)
    i += df.shape[0]
display(i)

1911704

Wall time: 3min 28s


In [13]:
%%time
filename = PATH + "full_players_games.csv"
df = pd.read_csv(filename, usecols=['match_id', 'radiant_win', 'duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire', 'first_blood_time'])

Wall time: 15.6 s


In [14]:
radiant_win = df['radiant_win'] == 't'
df = df.assign(radiant_win=radiant_win)

In [15]:
df

,match_id,radiant_win,duration,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire,first_blood_time
0,2304340261,True,1701,1975,4,63,3,100
1,2304335744,True,2299,1983,0,63,0,127
2,2304324185,True,3291,1584,0,60,0,196
3,2304339409,False,2192,0,1982,0,63,44
4,2304329004,True,2989,4,0,3,48,104
...,...,...,...,...,...,...,...,...
1911699,2306490763,False,2517,6,1972,3,63,15
1911700,2306496027,False,2100,256,2046,48,63,0
1911701,2306500208,True,1728,2047,0,63,0,56
1911702,2306480548,True,3274,1844,0,63,0,51


In [16]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [17]:
train, test = train_test_split(df, random_state=0)

In [18]:
train, val = train_test_split(train, random_state=0)

In [19]:
target = 'radiant_win'
features = ['duration', 'tower_status_radiant',
       'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire',
       'first_blood_time']
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

In [26]:
import numpy as np
pipeline = make_pipeline(
    SimpleImputer(),
    StandardScaler(),
    XGBClassifier(max_depth=10,
                  learning_rate=0.2,
                  booster='gbtree',
                  gamma=0.01,
                  n_jobs=-1,
                  random_state=0,
                  early_stopping_rounds=50,
                  verbosity=3
                 ),
    verbose=True
)
pipeline.fit(X_train, y_train)

Exception ignored in: <function Booster.__del__ at 0x00000257F2CC6A68>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py", line 957, in __del__
    if self.handle is not None:
AttributeError: 'Booster' object has no attribute 'handle'


[Pipeline] ..... (step 1 of 3) Processing simpleimputer, total=   0.2s
[Pipeline] .... (step 2 of 3) Processing standardscaler, total=   0.3s
[10:52:47] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 642 extra nodes, 4 pruned nodes, max_depth=10
[0]	validation_0-error:0.219856	validation_1-error:0.221897
[10:52:48] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 642 extra nodes, 0 pruned nodes, max_depth=10
[1]	validation_0-error:0.538102	validation_1-error:0.538222
[10:52:49] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 668 extra nodes, 2 pruned nodes, max_depth=10
[2]	validation_0-error:0.219847	validation_1-error:0.221869
[10:52:49] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 658 extra nodes, 0 pruned nodes, max_depth=10
[3]	validation_0-error:0.219847	validation_1-error:0.221869
[10:52:50] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 692 extra nodes, 0 pruned nodes, max_depth=10
[4]	validation_0-e

[10:53:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 288 extra nodes, 0 pruned nodes, max_depth=10
[45]	validation_0-error:0.223423	validation_1-error:0.225633
[10:53:21] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 276 extra nodes, 0 pruned nodes, max_depth=10
[46]	validation_0-error:0.223423	validation_1-error:0.225633
[10:53:21] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 486 extra nodes, 0 pruned nodes, max_depth=10
[47]	validation_0-error:0.223422	validation_1-error:0.225633
[10:53:22] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 236 extra nodes, 0 pruned nodes, max_depth=10
[48]	validation_0-error:0.223422	validation_1-error:0.225633
[10:53:23] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 278 extra nodes, 0 pruned nodes, max_depth=10
[49]	validation_0-error:0.223425	validation_1-error:0.225633
[10:53:24] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 348 extra nodes, 0 pr

[10:53:55] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 432 extra nodes, 2 pruned nodes, max_depth=10
[91]	validation_0-error:0.223422	validation_1-error:0.225636
[10:53:55] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 168 extra nodes, 0 pruned nodes, max_depth=10
[92]	validation_0-error:0.223422	validation_1-error:0.225636
[10:53:56] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 308 extra nodes, 0 pruned nodes, max_depth=10
[93]	validation_0-error:0.223422	validation_1-error:0.225636
[10:53:57] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 402 extra nodes, 2 pruned nodes, max_depth=10
[94]	validation_0-error:0.223422	validation_1-error:0.225636
[10:53:58] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 448 extra nodes, 2 pruned nodes, max_depth=10
[95]	validation_0-error:0.223422	validation_1-error:0.225636
[10:53:58] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 0 pr

Pipeline(memory=None,
         steps=[('simpleimputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, early_stopping_rounds=50,
                               gamma=0.01, learning_rate=0.2, max_delta_step=0,
                               max_depth=10, min_child_weight=1, missing=None,
                               n_estimators=100, n_jobs=-1, nthread=None,
                               objective='binary:logistic', random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
      

In [45]:
pipeline.score(X_val, y_val)

0.9948360278424863

In [46]:
pipeline.score(X_test, y_test)

0.9947878960341141

In [26]:
import xgboost

In [28]:
xgboost.__version__

'0.90'

In [30]:
import joblib
joblib.dump(pipeline, '../assets/whowon.pkl')

['../assets/whowon.pkl']

In [31]:
df.duration.min()

0

In [32]:
df.duration.max()

18232

In [38]:
df.duration.mean()

2535.112245933471

In [36]:
4500/60

75.0

In [41]:
for feature in features:
    print(feature)
    print(df[feature].min(), df[feature].mean() - df[feature].std() * 3, df[feature].mean(), df[feature].mean() + df[feature].std() * 3, df[feature].max())

duration
0 562.5081775897913 2535.112245933471 4507.716314277151 18232
tower_status_radiant
0 -1811.5530718552864 1013.2099038344849 3837.9728795242563 2047
tower_status_dire
0 -1836.7555235441628 958.0768685947197 3752.909260733602 2047
barracks_status_radiant
0 -51.27789014684418 36.29389644003465 123.86568302691347 63
barracks_status_dire
0 -48.918880978059384 36.636536827877116 122.19195463381362 63
first_blood_time
0 -169.13021613340408 108.92677893648808 386.9837740063802 2863


In [46]:
print(features)

['duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire', 'first_blood_time']


In [44]:
import eli5
from eli5.sklearn import PermutationImportance
import joblib
from xgboost import XGBClassifier
pipeline = joblib.load('../assets/whowon.pkl')
X_test_transformed = pipeline['standardscaler'].transform(pipeline['simpleimputer'].transform(X_test))
model = pipeline['xgbclassifier']

In [36]:
permuter = PermutationImportance(
    model,
    scoring='accuracy',
    n_iter=5,
    random_state=0
)

permuter.fit(X_test_transformed, y_test)

[10:58:20] ======== Monitor:  ========
[10:58:20] ======== Monitor:  ========


PermutationImportance(cv='prefit',
                      estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                              colsample_bylevel=1,
                                              colsample_bynode=1,
                                              colsample_bytree=1,
                                              early_stopping_rounds=50,
                                              gamma=0.01, learning_rate=0.2,
                                              max_delta_step=0, max_depth=10,
                                              min_child_weight=1, missing=nan,
                                              n_estimators=100, n_jobs=-1,
                                              nthread=None,
                                              objective='binary:logistic',
                                              random_state=0, reg_alpha=0,
                                              reg_lambda=1, scale_pos_weight=1,
               

In [37]:
feature_names = X_test.columns.tolist()
pd.Series(permuter.feature_importances_, feature_names).sort_values()

first_blood_time           0.001927
duration                   0.003187
barracks_status_radiant    0.005202
barracks_status_dire       0.007065
tower_status_dire          0.048610
tower_status_radiant       0.082302
dtype: float64

In [38]:
eli5.show_weights(
    permuter,
    top=None,
    feature_names=feature_names
)

Weight,Feature
0.0823 ± 0.0004,tower_status_radiant
0.0486 ± 0.0005,tower_status_dire
0.0071 ± 0.0001,barracks_status_dire
0.0052 ± 0.0002,barracks_status_radiant
0.0032 ± 0.0002,duration
0.0019 ± 0.0001,first_blood_time


In [39]:
joblib.dump(permuter, '../assets/permuter.pkl')

['../assets/permuter.pkl']

In [41]:
print(feature_names)

['duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire', 'first_blood_time']


In [43]:
eli5.show_weights(
    permuter,
    top=None,
    feature_names=feature_names
)

Weight,Feature
0.0823 ± 0.0004,tower_status_radiant
0.0486 ± 0.0005,tower_status_dire
0.0071 ± 0.0001,barracks_status_dire
0.0052 ± 0.0002,barracks_status_radiant
0.0032 ± 0.0002,duration
0.0019 ± 0.0001,first_blood_time


In [47]:
eli5.__version__

'0.10.1'